In [128]:
import scripts
import numpy as np
import random
import time
import csv
import pickle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import matplotlib.pyplot as plt
from scripts import helpers

In [5]:
data = np.loadtxt('grid_800_400_0_0/grid_data_0_0.csv', dtype = float, delimiter = ',')
M = pickle.load(open('grid_800_400_0_0/cardinality_pickle.pkl', 'rb'))
m00 = M[0][0]
sorted_m00 = sorted(m00.items(), cmp = lambda x, y: cmp(x[1], y[1]), reverse = True)
mask = np.array(map(lambda x: m00[x] > 20, data[:, 5]))
masked_data = data[mask, :]

In [16]:
print len(filter(lambda x: x[1] > 20, sorted_m00))
print sorted_m00[100]
print sorted_m00[200]
print sorted_m00[300]
print sorted_m00[400]
print sorted_m00[500]
print sorted_m00[600]

454

In [136]:
print masked_data.shape
print data.shape
(96039)*100/100348

(96039, 6)
(100348, 6)


95

In [146]:
y = masked_data[:, 5].astype(int)

label_encoder = LabelEncoder()
label_encoder.fit(y)

le_y = label_encoder.transform(y)
le_y = le_y.reshape(-1, 1)

oh_encoder = OneHotEncoder()
oh_encoder.fit(le_y)
oh_y = oh_encoder.transform(le_y)

Y = le_y
X = masked_data[:, (1,2,3,4)]

days = lambda t: (int(t) % (86400*7))/ 86400
hours = lambda t: (int(t) % 86400) / 3600

days_v = np.array(map(days, X[:, 3])).reshape(-1, 1)
hours_v = np.array(map(hours, X[:, 3])).reshape(-1, 1)

main_x = np.hstack((X[:, (0, 1, 2)], days_v, hours_v))

In [129]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=29, weights='distance', metric='manhattan')
clf.fit(main_x, Y)

pred_Y_prob = clf.predict_proba(main_x)
pred_labels = label_encoder.inverse_transform(np.argsort(pred_Y_prob, axis=1)[:,::-1][:,:3]) 
apk_arr = np.hstack((masked_data[:, -1].reshape(-1, 1), pred_labels))
apk_arr = apk_arr.astype(int)

apk_row = lambda r: helpers.apk(r[:1], r[1:])
t_apk = map(apk_row, apk_arr)
print np.mean(t_apk)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


1.0


In [130]:
pickle.dump(clf, open('../test_model.pkl', 'wb'))

In [132]:
mo = pickle.load(open('test_model.pkl', 'rb'))

In [133]:
mo_preds = mo.predict_proba(main_x)

In [141]:
import itertools
a = range(0, 13)
b = range(0, 24)

c = list(itertools.product(a, b))
print c[:5]
len(c)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]


312

In [140]:
c = list(c)

In [142]:
a = object()

In [143]:
a.name = "Anil"
a.age = 21

AttributeError: 'object' object has no attribute 'name'

In [42]:
a

NameError: name 'a' is not defined

In [148]:
a = 1
b = 2
print "printing %s %s" %(a, b)

printing 1 2


In [149]:
np.hstack?

In [150]:
pred_labels = label_encoder.inverse_transform(np.argsort(pred_Y_prob, axis=1)[:,::-1][:,:3]) 

In [151]:
pred_labels.shape

(96039, 3)

In [152]:
masked_data[:, 1].shape

(96039,)

In [157]:
temp = np.hstack((masked_data[:, 1].reshape(-1, 1), pred_labels))

In [160]:
 ','.join([1,2,3])

TypeError: sequence item 0: expected string, int found

In [61]:
import base_scikit_learn_model as sk
import grid_generation as grid
import numpy as np

In [ ]:
reload(grid)
reload(sk)
g = grid.Grid(files_flag=True)
sk_model = sk.SklearnModel(cross_validation_file='../main_cv_0.02_5.csv', test_file='../test.csv', grid = g)
sk_model.train()

Training 0, 0 grid
Time taken to train grid 0, 0 is: 2.10147714615
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Training 0, 1 grid
Time taken to train grid 0, 1 is: 2.56615710258
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Training 0, 2 grid
Time taken to train grid 0, 2 is: 2.48254799843
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Training 0, 3 grid
Time taken to train grid 0, 3 is: 2.41153001785
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Training 0, 4 grid
Time taken to train grid 0, 4 is: 2.7758898735
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Training 0, 5 grid
Time taken to train grid 0, 5 is: 2.59506297112
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Training 0, 6 grid
Time taken to train grid 0, 6 is: 2.30578708649
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Training 0, 7 grid
Time taken to train grid 0, 7 is: 2.28249788284
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Training 0, 8 grid
Time taken to train grid 0, 8 is: 2.65045619011
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Training 0,

In [92]:
test_data = np.loadtxt('../grid_800_400_200_100/grid_data_0_0.csv', dtype = float, delimiter = ',')

In [103]:
grid_wise_data = [[[] for n in range(26)] for m in range(13)]
for i in range(len(test_data)):
    m, n = sk.get_grids_of_a_point((test_data[i][1], test_data[i][2]), sk_model.grid)[0]
    grid_wise_data[m][n].append(test_data[i])

predictions = []
for m in range(13):
    for n in range(26):
        if len(grid_wise_data[m][n]) == 0:
            continue
        else:
            predictions.append(sk_model.predict_grid(np.array(grid_wise_data[m][n]), m, n))

In [105]:
for m in range(13):
    for n in range(26):
        if len(grid_wise_data[m][n]) > 0:
            print len(grid_wise_data[m][n]), m, n

100348 0 0
22484 0 1
17006 1 0
6323 1 1


In [111]:
predictions = np.vstack(tuple(predictions))
sorted_pre = predictions[predictions[:, 0].argsort()].astype(int)
sorted_pre[:5]

array([[       132, 5348440074, 9988088517, 4048573921],
       [       142, 5747062935, 4914943817, 8945520545],
       [       149, 8050782732, 9988088517, 4067911129],
       [       171, 5938943847, 9988088517, 4067911129],
       [       255, 1412889338, 9988088517, 4251632872]])

In [82]:
preds1 = sk_model.predict_grid(test_data, 0, 25)

In [85]:
preds1[0]

array([  1.32000000e+02,   5.34844007e+09,   9.98808852e+09,
         4.04857392e+09])

In [57]:
tt = test_data[:, 0].astype(int)

In [58]:
tt[:5]

array([132, 142, 149, 171, 255])

In [ ]:
a = [1,2,3]
a[-1:]

In [66]:
import grid_generation as grid
grid.get_grids((10.0000, 10.0000), 800, 400, 200, 100)

[(12, 25)]

In [77]:
[[1,2,3]]*3

[[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [113]:
sk_model.check_cross_validation()

TypeError: hstack() takes exactly 1 argument (2 given)